## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-09-17-42-04 +0000,nyt,Israel Launches New Ground Incursion in Lebano...,https://www.nytimes.com/2025/07/09/world/middl...
1,2025-07-09-17-34-43 +0000,bbc,Linda Yaccarino departs as boss of Musk's X,https://www.bbc.com/news/articles/cx2gy3j9xq6o
2,2025-07-09-17-30-19 +0000,bbc,Greece suspends asylum applications for migran...,https://www.bbc.com/news/articles/cx2nx0ey7wyo
3,2025-07-09-17-20-25 +0000,nyt,"Japan, Moldova, Iraq: Here Are Trump’s New Tar...",https://www.nytimes.com/2025/07/07/business/tr...
4,2025-07-09-17-13-16 +0000,bbc,Police preparing for Donald Trump to visit Sco...,https://www.bbc.com/news/articles/cm20xvgmd8yo


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2291/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
25,trump,14
2,new,4
28,police,4
37,texas,4
83,deal,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
29,2025-07-09-12-26-31 +0000,bbc,'Lives are at stake:' Deadly Texas storms put ...,https://www.bbc.com/news/articles/cx23ye0xnnyo,29
3,2025-07-09-17-20-25 +0000,nyt,"Japan, Moldova, Iraq: Here Are Trump’s New Tar...",https://www.nytimes.com/2025/07/07/business/tr...,26
54,2025-07-08-23-01-40 +0000,bbc,Faisal Islam: Trump delays tariffs as the rest...,https://www.bbc.com/news/articles/c628nk51364o,26
25,2025-07-09-13-17-51 +0000,nyt,Trump Treats Tariffs More as a Form of Power T...,https://www.nytimes.com/2025/07/09/us/politics...,25
36,2025-07-09-10-54-34 +0000,nyt,Trump Takes Reins on U.S. Economy With Policy ...,https://www.nytimes.com/2025/07/09/us/politics...,25


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
29,29,2025-07-09-12-26-31 +0000,bbc,'Lives are at stake:' Deadly Texas storms put ...,https://www.bbc.com/news/articles/cx23ye0xnnyo
40,20,2025-07-09-09-32-29 +0000,bbc,At least three dead in New Mexico flash flooding,https://www.bbc.com/news/articles/cx2jk51xv65o
35,16,2025-07-09-10-55-24 +0000,nyt,Jailed Kurdish Leader Declares the End to PKK’...,https://www.nytimes.com/2025/07/09/world/middl...
22,14,2025-07-09-13-51-41 +0000,bbc,French police raid on National Rally HQ prompt...,https://www.bbc.com/news/articles/ckg5kd04e1jo
11,13,2025-07-09-16-06-35 +0000,bbc,EU hopes to agree US tariff deal 'in coming days',https://www.bbc.com/news/articles/cev0n01y8wwo
10,13,2025-07-09-16-28-07 +0000,nyt,Israel’s Deadly Assault on Iran Prison Incites...,https://www.nytimes.com/2025/07/06/world/middl...
60,13,2025-07-08-18-35-46 +0000,bbc,Why small-time criminals burned a London wareh...,https://www.bbc.com/news/articles/czjkke22gv9o
54,12,2025-07-08-23-01-40 +0000,bbc,Faisal Islam: Trump delays tariffs as the rest...,https://www.bbc.com/news/articles/c628nk51364o
9,11,2025-07-09-16-29-27 +0000,bbc,Bin lorry drivers face axe as strike talks fail,https://www.bbc.com/news/articles/crk6rz4lg20o
8,10,2025-07-09-16-40-00 +0000,bbc,UK must refuse to take male small boat migrant...,https://www.bbc.com/news/articles/c4g8rqlxn2vo


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
